# GenAI with Python: AI Agents with Multi-Session Memory

###### [Read the Article on TDS]()

### 0 - Setup

In [1]:
#pip install ollama==0.5.1
import ollama
llm = "qwen2.5"

In [2]:
stream = ollama.generate(model=llm, prompt='''Do you remember what i asked you last time?''', stream=True)
for chunk in stream:
    print(chunk['response'], end='', flush=True)

Hello! I'm here to help, but I don't have access to previous interactions or conversations. Could you please let me know what you'd like assistance with today? If you had a specific question before, feel free to ask it again or provide more context so we can get started.

### 1 - DB

In [2]:
#pip install chromadb==0.5.23
import chromadb

db = chromadb.PersistentClient()
db.list_collections()

[Collection(name=chat_history)]

In [4]:
collection_name = "chat_history"
collection = db.get_or_create_collection(name=collection_name, 
                                         embedding_function=chromadb.utils.embedding_functions.DefaultEmbeddingFunction())

In [6]:
from datetime import datetime

def save_chat(lst_msg, collection):
    print("--- Saving ---")
    ## extract chat
    chat = ""
    for m in lst_msg:
        chat += f'{m["role"]}: <<{m["content"]}>>' +'\n\n'
    ## get idx
    idx = str(collection.count() +1)
    ## generate info
    p = "Describe the following conversation using only 3 keywords separated by a comma (for example: 'finance, volatility, stocks')."
    tags = ollama.generate(model=llm, prompt=p+"\n"+chat)["response"]
    dic_info = {"tags":tags,
                "date": datetime.today().strftime("%Y-%m-%d"),
                "time": datetime.today().strftime("%H:%M")}
    ## write db
    collection.add(documents=[chat], ids=[idx], metadatas=[dic_info])
    print(f"--- Chat num {idx} saved ---","\n")
    print(dic_info,"\n")
    print(chat)
    print("------------------------")

### 2 - Run Basic Agent

In [9]:
prompt = "You are an intelligent assistant, provide the best possible answer to user's request."
messages = [{"role":"system", "content":prompt}]

while True:    
    ## User
    q = input('🙂 >')
    if q == "quit":
        ### save chat before quitting
        save_chat(lst_msg=messages, collection=collection)
        break
    messages.append( {"role":"user", "content":q} )
   
    ## Model
    agent_res = ollama.chat(model=llm, messages=messages, tools=[])
    res = agent_res["message"]["content"]
   
    ## Response
    print("👽 >", f"\x1b[1;30m{res}\x1b[0m")
    messages.append( {"role":"assistant", "content":res} )

🙂 > hi


👽 > Hello! How can I assist you today?


🙂 > i must tell you that im highly allergic to chocolate. i could die


👽 > I'm really sorry to hear that. It's important to take this allergy very seriously. Here are some steps you should consider:

1. **Inform Your Healthcare Provider**: Make sure your doctor and any healthcare professionals who might be involved in treating you are aware of your severe chocolate allergy.

2. **Carry Medication**: Always carry an epinephrine auto-injector (like an EpiPen) if prescribed by a doctor, and know how to use it.

3. **Avoid Triggers**: Be vigilant about avoiding any products that contain chocolate or cocoa.

4. **Read Labels Carefully**: Even items you think wouldn’t have chocolate can sometimes contain traces due to cross-contamination.

5. **Emergency Plan**: Have an emergency plan in place, including knowing which hospitals are best equipped to handle severe allergic reactions and who your key contacts are during an emergency.

6. **Education**: Educate those around you about the severity of your allergy so they know what to do if a reaction occurs.

If you

🙂 > quit


--- Chat num 1 saved --- 

{'tags': 'allergy, emergency, avoidance', 'date': '2025-06-24', 'time': '23:08'} 

system: <<You are an intelligent assistant, provide the best possible answer to user's request.>>

user: <<hi>>

assistant: <<Hello! How can I assist you today?>>

user: <<i must tell you that im highly allergic to chocolate. i could die>>

assistant: <<I'm really sorry to hear that. It's important to take this allergy very seriously. Here are some steps you should consider:

1. **Inform Your Healthcare Provider**: Make sure your doctor and any healthcare professionals who might be involved in treating you are aware of your severe chocolate allergy.

2. **Carry Medication**: Always carry an epinephrine auto-injector (like an EpiPen) if prescribed by a doctor, and know how to use it.

3. **Avoid Triggers**: Be vigilant about avoiding any products that contain chocolate or cocoa.

4. **Read Labels Carefully**: Even items you think wouldn’t have chocolate can sometimes contain tra

### 3 - Tools

In [9]:
def retrieve_chat(query:str) -> str:
    res_db = collection.query(query_texts=[query])["documents"][0][0:10]
    history = ' '.join(res_db).replace("\n", " ")
    return history

tool_retrieve_chat = {'type':'function', 'function':{
  'name': 'retrieve_chat',
  'description': 'When you knowledge is NOT enough to answer the user, you can use this tool to retrieve chats history.',
  'parameters': {'type': 'object', 
                 'required': ['query'],
                 'properties': {
                    'query': {'type':'str', 'description':'Input the user question or the topic of the current chat'},
}}}}
## test
retrieve_chat(query="choco")

Number of requested results 10 is greater than number of elements in index 1, updating n_results = 1


"system: <<You are an intelligent assistant, provide the best possible answer to user's request.>>  user: <<hi>>  assistant: <<Hello! How can I assist you today?>>  user: <<i must tell you that im highly allergic to chocolate. i could die>>  assistant: <<I'm really sorry to hear that. It's important to take this allergy very seriously. Here are some steps you should consider:  1. **Inform Your Healthcare Provider**: Make sure your doctor and any healthcare professionals who might be involved in treating you are aware of your severe chocolate allergy.  2. **Carry Medication**: Always carry an epinephrine auto-injector (like an EpiPen) if prescribed by a doctor, and know how to use it.  3. **Avoid Triggers**: Be vigilant about avoiding any products that contain chocolate or cocoa.  4. **Read Labels Carefully**: Even items you think wouldn’t have chocolate can sometimes contain traces due to cross-contamination.  5. **Emergency Plan**: Have an emergency plan in place, including knowing wh

In [10]:
def final_answer(text:str) -> str:
    return text

tool_final_answer = {'type':'function', 'function':{
  'name': 'final_answer',
  'description': 'Returns a natural language response to the user',
  'parameters': {'type': 'object', 
                 'required': ['text'],
                 'properties': {'text': {'type':'str', 'description':'natural language response'}}
}}}

In [11]:
dic_tools = {'retrieve_chat':retrieve_chat, 'final_answer':final_answer}

### 4 - Run Agent with Memory

In [13]:
# Utils
def use_tool(agent_res:dict, dic_tools:dict) -> dict:
    ## use tool
    if agent_res["message"].tool_calls is not None:
        for tool in agent_res["message"].tool_calls:
            t_name, t_inputs = tool["function"]["name"], tool["function"]["arguments"]
            if f := dic_tools.get(t_name):
                ### calling tool
                print('🔧 >', f"\x1b[1;31m{t_name} -> Inputs: {t_inputs}\x1b[0m")
                ### tool output
                t_output = f(**tool["function"]["arguments"])
                print(t_output)
                ### final res
                res = t_output
            else:
                print('🤬 >', f"\x1b[1;31m{t_name} -> NotFound\x1b[0m")      
    ## don't use tool
    else:
        res = agent_res["message"].content
        t_name, t_inputs = '', ''
    return {'res':res, 'tool_used':t_name, 'inputs_used':t_inputs}

In [14]:
def run_agent(llm, messages, available_tools):
    ## use tools until final answer
    tool_used, local_memory = '', ''
    while tool_used != 'final_answer':
        ### use tool
        try:
            agent_res = ollama.chat(model=llm, messages=messages, tools=[v for v in available_tools.values()])
            dic_res = use_tool(agent_res, dic_tools)
            res, tool_used, inputs_used = dic_res["res"], dic_res["tool_used"], dic_res["inputs_used"]
        ### error
        except Exception as e:
            print("⚠️ >", e)
            res = f"I tried to use {tool_used} but didn't work. I will try something else."
            print("👽 >", f"\x1b[1;30m{res}\x1b[0m")
            messages.append( {"role":"assistant", "content":res} )       
        ### update memory
        if tool_used not in ['','final_answer']:
            local_memory += f"\n{res}"
            messages.append( {"role":"assistant", "content":local_memory} )
            available_tools.pop(tool_used)
            if len(available_tools) == 1:
                messages.append( {"role":"user", "content":"now activate the tool final_answer."} ) 
        ### tools not used
        if tool_used == '':
            break
    return res

In [18]:
prompt = '''
You are an intelligent assistant, provide the best possible answer to user's request.
You must return natural language response.
When the user gives a task, first use the tool 'retrieve_chat' to retrieve older information.  
'''
messages = [{"role":"system", "content":prompt}]

while True:
    ## User
    q = input('🙂 >')
    if q == "quit":
        ### save chat before quitting
        save_chat(lst_msg=messages, collection=collection)
        break
    messages.append( {"role":"user", "content":q} )
   
    ## Model
    available_tools = {"retrieve_chat":tool_retrieve_chat, "final_answer":tool_final_answer}
    res = run_agent(llm, messages, available_tools)
   
    ## Response
    print("👽 >", f"\x1b[1;30m{res}\x1b[0m")
    messages.append( {"role":"assistant", "content":res} )

🙂 > hi


👽 > Hello! How can I assist you today?


🙂 > i want to eat the famous Vienna sacher torte. Give me the recipe


Number of requested results 10 is greater than number of elements in index 1, updating n_results = 1


🔧 > retrieve_chat -> Inputs: {'query': 'recipe for sacher torte'}
system: <<You are an intelligent assistant, provide the best possible answer to user's request.>>  user: <<hi>>  assistant: <<Hello! How can I assist you today?>>  user: <<i must tell you that im highly allergic to chocolate. i could die>>  assistant: <<I'm really sorry to hear that. It's important to take this allergy very seriously. Here are some steps you should consider:  1. **Inform Your Healthcare Provider**: Make sure your doctor and any healthcare professionals who might be involved in treating you are aware of your severe chocolate allergy.  2. **Carry Medication**: Always carry an epinephrine auto-injector (like an EpiPen) if prescribed by a doctor, and know how to use it.  3. **Avoid Triggers**: Be vigilant about avoiding any products that contain chocolate or cocoa.  4. **Read Labels Carefully**: Even items you think wouldn’t have chocolate can sometimes contain traces due to cross-contamination.  5. **Emerge

🙂 > oh right, thank you. then invent a version of it without chocolate


🔧 > final_answer -> Inputs: {'text': 'Here’s a recipe for an allergy-friendly version of the Sacher Torte: Use a light sponge cake base and serve it with a rich apricot filling. The ingredients include 200g unsalted butter, 175g granulated sugar, 4 large eggs, 1 tsp vanilla extract, pinch of salt for the sponge cake; and 300g dried apricots, 125ml water, 50g unsalted butter, 1 tsp lemon zest, 3 tbsp cornstarch for the filling. Follow the instructions to create a delightful treat.'}
Here’s a recipe for an allergy-friendly version of the Sacher Torte: Use a light sponge cake base and serve it with a rich apricot filling. The ingredients include 200g unsalted butter, 175g granulated sugar, 4 large eggs, 1 tsp vanilla extract, pinch of salt for the sponge cake; and 300g dried apricots, 125ml water, 50g unsalted butter, 1 tsp lemon zest, 3 tbsp cornstarch for the filling. Follow the instructions to create a delightful treat.
👽 > Here’s a recipe for an allergy-friendly version of the Sacher 

🙂 > quit


--- Saving ---
--- Chat num 2 saved --- 

{'tags': 'sugar, eggs, apricot', 'date': '2025-06-25', 'time': '21:06'} 

system: <<
You are an intelligent assistant, provide the best possible answer to user's request.
You must return natural language response.
When the user gives a task, first use the tool 'retrieve_chat' to retrieve older information.  
>>

user: <<hi>>

assistant: <<Hello! How can I assist you today?>>

user: <<i want to eat the famous Vienna sacher torte. Give me the recipe>>

assistant: <<
system: <<You are an intelligent assistant, provide the best possible answer to user's request.>>  user: <<hi>>  assistant: <<Hello! How can I assist you today?>>  user: <<i must tell you that im highly allergic to chocolate. i could die>>  assistant: <<I'm really sorry to hear that. It's important to take this allergy very seriously. Here are some steps you should consider:  1. **Inform Your Healthcare Provider**: Make sure your doctor and any healthcare professionals who might be in

In [19]:
messages

[{'role': 'system',
  'content': "\nYou are an intelligent assistant, provide the best possible answer to user's request.\nYou must return natural language response.\nWhen the user gives a task, first use the tool 'retrieve_chat' to retrieve older information.  \n"},
 {'role': 'user', 'content': 'hi'},
 {'role': 'assistant', 'content': 'Hello! How can I assist you today?'},
 {'role': 'user',
  'content': 'i want to eat the famous Vienna sacher torte. Give me the recipe'},
 {'role': 'assistant',
  'content': "\nsystem: <<You are an intelligent assistant, provide the best possible answer to user's request.>>  user: <<hi>>  assistant: <<Hello! How can I assist you today?>>  user: <<i must tell you that im highly allergic to chocolate. i could die>>  assistant: <<I'm really sorry to hear that. It's important to take this allergy very seriously. Here are some steps you should consider:  1. **Inform Your Healthcare Provider**: Make sure your doctor and any healthcare professionals who might 